In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn as sk
from sklearn.preprocessing import StandardScaler   #for scaling
from sklearn.model_selection import train_test_split  #for data splitting
from sklearn.neural_network import MLPClassifier      #classifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/ee-769-assignment1/test.csv
/kaggle/input/ee-769-assignment1/sample_submission.csv
/kaggle/input/ee-769-assignment1/train.csv


In [2]:
#reading data and preparing data
Train=pd.read_csv("../input/ee-769-assignment1/train.csv") 
Test=pd.read_csv("../input/ee-769-assignment1/test.csv") 
print(Train.shape)
print(Test.shape)


(1028, 34)
(441, 33)


In [3]:
def pre_processing(X):                                        #preprocessing function
  obj_df = X.select_dtypes(include=['object']).copy()         #select object data
  C=obj_df.columns
  X=pd.concat([X,pd.get_dummies(obj_df)],axis=1)              #one hot vector
  X.drop(C, axis=1, inplace=True)                             #remove object data
  X.drop(['Attrition'],axis=1, inplace=True,errors='ignore')  #remove output if exixst
  C=X.columns
  scaler = MinMaxScaler(feature_range=(0, 1))
  for c in C:
  	X[[c]]=scaler.fit_transform(X[[c]])  
  return X

In [4]:
Y=Train.Attrition
X=Train.loc[:,Train.columns !='Attrition']
X=X.loc[:,X.columns !='EmployeeCount']
X=X.loc[:,X.columns !='EmployeeNumber']
X=X.loc[:,X.columns !='ID']
X=X.loc[:,X.columns !='DailyRate']
X=X.loc[:,X.columns !='HourlyRate']
X=X.loc[:,X.columns !='MonthlyRate']


X=pre_processing(X) 

In [5]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.15,random_state=44)     #splitting data

print("training shape",X_train.shape)
print("testing shape",X_test.shape)

clf = MLPClassifier(hidden_layer_sizes=(35,35,35),               #classifier parameters
                    max_iter=500,activation = 'relu',
                    alpha=3,solver='adam',
                    learning_rate_init=0.005,
                    random_state=7)

#clf = KNeighborsClassifier(n_neighbors=7,p=2)
clf.fit(X_train,Y_train)                                    #train the model

score = clf.score(X_train, Y_train)
print("accuracy on train set",score)
score = clf.score(X_test, Y_test)
print("accuracy on validation set",score)

t=clf.predict(X_test)                                    #model prediction vector
cnf_matrix = metrics.confusion_matrix(Y_test, t)         #confusion matrix
print(cnf_matrix)                              

training shape (873, 48)
testing shape (155, 48)
accuracy on train set 0.8843069873997709
accuracy on validation set 0.8903225806451613
[[128   1]
 [ 16  10]]


In [6]:
#for submission
X_t=Test.loc[:,Test.columns !='EmployeeCount']
X_t=X_t.loc[:,X_t.columns !='EmployeeNumber']
Id=X_t.ID
X_t=X_t.loc[:,X_t.columns !='ID']
X_t=X_t.loc[:,X_t.columns !='DailyRate']
X_t=X_t.loc[:,X_t.columns !='HourlyRate']
X_t=X_t.loc[:,X_t.columns !='MonthlyRate']

X_t=pre_processing(X_t) 


print("modified input shpe",X_t.shape)
X_t.head()

output=clf.predict(X_t)
op={'ID':Id,'Attrition':output}
op=pd.DataFrame(op)
op.to_csv('output.csv',index=False)

modified input shpe (441, 48)
